## 3. Model Training & Evaluation

Now that we have our data prepared and augmented, let's train a YOLO model for palm detection.

### What This Training Code Does:

**1. Model Loading (`YOLO('yolov8n.pt')`):**
- Downloads and loads YOLOv8 Nano (smallest, fastest version)
- Pre-trained on COCO dataset (general object detection)
- Will be fine-tuned for your palm detection task

**2. Training (`model.train()`):**
- **data**: Points to your dataset configuration file
- **epochs=50**: Trains for 50 complete passes through your data
- **imgsz=640**: Resizes images to 640x640 pixels
- **batch=16**: Processes 16 images at once (adjust based on GPU memory)
- **name**: Saves results with this experiment name
- **device=0**: Uses GPU 0 (change to 'cpu' if no GPU available)

**3. Evaluation (`model.val()`):**
- Tests the trained model on validation set
- Returns metrics like mAP (mean Average Precision), precision, recall

**4. Prediction & Visualization:**
- Runs inference on validation images
- Saves results with bounding boxes drawn
- **conf=0.5**: Only shows detections with >50% confidence

In [1]:
# YOLO Model Loading - High Performance Setup
from ultralytics import YOLO
import torch

# Choose model based on your laptop or pc performance
# YOLOv8n = fastest but lower accuracy (around 85-90%)
# YOLOv8s = balanced speed and accuracy (around 90-95%)
# YOLOv8m = higher accuracy but needs more memory (95%+)

# Use YOLOv8s for better accuracy but still okay for normal laptop
model_name = 'yolov8n.pt'  # change to yolov8s.pt if your laptop can handle it

print(f"Loading {model_name} (small model for good accuracy)...")
model = YOLO(model_name)  # will auto download if not available

print("Model loaded successfully!")
print(f"Model parameters: ~{sum(p.numel() for p in model.model.parameters())/1e6:.1f}M")

# check gpu memory before training
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    if gpu_memory < 3:  # if gpu less than 3gb memory
        print("Low GPU memory detected. Use YOLOv8n if training fails.")
        print("To switch: change 'yolov8s.pt' to 'yolov8n.pt' above")


Loading yolov8n.pt (Small model for higher accuracy)...
Model loaded successfully!
Model parameters: ~3.2M


In [2]:
# pre-training checks + laptop-friendly tips
import os
import torch
import psutil

def check_training_setup():
    """quick check: dataset paths ok, count images, show system specs, give simple training advice"""
    issues = []
    recommendations = []
    
    # check dataset yaml exists
    dataset_path = 'datasets/palms_yolo/dataset.yaml'
    if not os.path.exists(dataset_path):
        issues.append(f"❌ Dataset config not found: {dataset_path}")
    
    # check image dirs exist and count files - UPDATED to check train_aug
    dirs_to_check = [
        'datasets/palms_yolo/train_aug/images',  # Updated to use augmented data
        'datasets/palms_yolo/valid/images',
        'datasets/palms_yolo/test/images'
    ]
    
    total_train_images = 0
    for dir_path in dirs_to_check:
        if not os.path.exists(dir_path):
            issues.append(f"❌ Directory missing: {dir_path}")
        else:
            img_count = len([f for f in os.listdir(dir_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            print(f"✅ {dir_path}: {img_count} images")
            if 'train_aug' in dir_path:  # Updated to check train_aug
                total_train_images = img_count
    
    # show basic system info (ram, cpu)
    ram_gb = psutil.virtual_memory().total / (1024**3)
    cpu_count = psutil.cpu_count()
    print(f"\nSystem Specs:")
    print(f"   RAM: {ram_gb:.1f} GB")
    print(f"   CPU Cores: {cpu_count}")
    
    # gpu info + simple batch size hint
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"   GPU: {gpu_name}")
        print(f"   GPU Memory: {gpu_memory:.1f} GB")
        
        # batch size suggestion by vram
        if gpu_memory < 4:
            recommendations.append("Low GPU memory: Use batch_size=4-8")
        elif gpu_memory < 8:
            recommendations.append("Medium GPU memory: Use batch_size=8-16")
        else:
            recommendations.append("High GPU memory: Use batch_size=16-32")
            
    else:
        print("   GPU: None (CPU training)")
        recommendations.append("CPU training: Use batch_size=2-4, reduce epochs")
    
    # advice based on how many train images - UPDATED thresholds for augmented data
    if total_train_images < 500:
        recommendations.append("📊 Small dataset: Use heavy augmentation + more epochs (150-200)")
    elif total_train_images < 800:  # Updated threshold
        recommendations.append("📊 Medium dataset: Use moderate augmentation + 100-150 epochs")
    else:
        recommendations.append("📊 Large dataset: Standard training with 80-120 epochs")
    
    # model size suggestion by gpu memory
    if torch.cuda.is_available() and torch.cuda.get_device_properties(0).total_memory > 4e9:
        recommendations.append("🎯 For 98% accuracy: Use YOLOv8s or YOLOv8m with augmented data")
    else:
        recommendations.append("🎯 Laptop-friendly: Start with YOLOv8s on augmented data, fallback to YOLOv8n if needed")
    
    # Enhanced recommendation for augmented data
    if total_train_images > 500:
        recommendations.append(f"🚀 AUGMENTED DATA: {total_train_images} images should reach 97-98% accuracy!")
    
    # print any problems found
    if issues:
        print("\n🚨 Issues found:")
        for issue in issues:
            print(f"  {issue}")
    
    # print simple tips
    if recommendations:
        print("\n💡 Recommendations for 98% target:")
        for rec in recommendations:
            print(f"  {rec}")
    
    # ready signal if no issues
    if not issues:
        print("\n🎉 Setup ready for high-performance training with augmented data!")
        return True
    return False

# run the checks
check_training_setup()

✅ datasets/palms_yolo/train/images: 338 images
✅ datasets/palms_yolo/valid/images: 22 images
✅ datasets/palms_yolo/test/images: 64 images

System Specs:
   RAM: 7.8 GB
   CPU Cores: 4
   GPU: None (CPU training)

💡 Recommendations for 95% target:
  CPU training: Use batch_size=2-4, reduce epochs
  📊 Small dataset: Use heavy augmentation + more epochs (150-200)
  🎯 Laptop-friendly: Start with YOLOv8s, fallback to YOLOv8n if needed

🎉 Setup ready for high-performance training!


True

In [3]:
# Check if model is already trained
import os
from glob import glob

def check_existing_training():
    """Check if there are any existing training results"""
    
    print("🔍 Checking for existing trained models...")
    
    # Check for YOLO training results
    runs_detect = "runs/detect"
    if os.path.exists(runs_detect):
        experiments = [d for d in os.listdir(runs_detect) if os.path.isdir(os.path.join(runs_detect, d))]
        if experiments:
            print(f"✅ Found {len(experiments)} previous training experiments:")
            for exp in experiments:
                exp_path = os.path.join(runs_detect, exp)
                weights_path = os.path.join(exp_path, "weights")
                if os.path.exists(weights_path):
                    weights = os.listdir(weights_path)
                    print(f"   📁 {exp}: {weights}")
                else:
                    print(f"   📁 {exp}: No weights found")
            
            # Check for best.pt (trained model)
            latest_exp = max(experiments, key=lambda x: os.path.getctime(os.path.join(runs_detect, x)))
            best_model_path = os.path.join(runs_detect, latest_exp, "weights", "best.pt")
            
            if os.path.exists(best_model_path):
                print(f"\n🎯 TRAINED MODEL FOUND!")
                print(f"   Location: {best_model_path}")
                print(f"   You can load it with: model = YOLO('{best_model_path}')")
                return True, best_model_path
            else:
                print(f"\n⚠️ Training folders exist but no trained model (best.pt) found")
                return False, None
        else:
            print("❌ No training experiments found")
            return False, None
    else:
        print("❌ No training results directory found")
        return False, None

# Check training status
has_trained_model, model_path = check_existing_training()

if has_trained_model:
    print(f"\n✅ YOUR MODEL IS ALREADY TRAINED!")
    print(f"📊 You can skip the training step and go directly to evaluation")
    print(f"🔄 To use the trained model, run: model = YOLO('{model_path}')")
else:
    print(f"\n❌ NO TRAINED MODEL FOUND")
    print(f"🚀 You need to run the training cells to train your model")

🔍 Checking for existing trained models...
✅ Found 7 previous training experiments:
   📁   Palm_Training: []
   📁 Palm_Training: []
   📁 Palm_Training2: ['best.pt', 'epoch0.pt', 'epoch20.pt', 'epoch40.pt', 'last.pt']
   📁 val: No weights found
   📁 val2: No weights found
   📁 val3: No weights found
   📁 YOLOv8n_Optimized_98pct_Target: ['best.pt', 'epoch0.pt', 'epoch10.pt', 'epoch20.pt', 'epoch30.pt', 'epoch40.pt', 'epoch50.pt', 'last.pt']

⚠️ Training folders exist but no trained model (best.pt) found

❌ NO TRAINED MODEL FOUND
🚀 You need to run the training cells to train your model


In [4]:
# Laptop-Optimized Training Configuration for Augmented Data
print("💻 Starting laptop-optimized training with AUGMENTED dataset...")
print("Target: 97-98% accuracy in 8-10 hours")
print("⚡ Using 616 augmented images (82% more data!)")

def estimate_laptop_training_time():
    """Estimate training time for optimized configuration with augmented data"""
    
    # Get training images count from augmented directory
    train_dir = 'datasets/palms_yolo/train_aug/images'  # Updated to use train_aug
    if os.path.exists(train_dir):
        train_images = len([f for f in os.listdir(train_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    else:
        train_images = 616  # Expected augmented dataset size
    
    # Optimized parameters
    epochs = 80
    batch_size = 4  # Increased from 2 for better efficiency
    img_size = 416  # Reduced from 640
    
    # Time calculation for CPU training with optimizations
    steps_per_epoch = train_images // batch_size
    seconds_per_step = 1.8  # Faster due to smaller images and model
    
    epoch_time = steps_per_epoch * seconds_per_step
    total_hours = (epochs * epoch_time) / 3600
    
    print(f"\n⏱️ AUGMENTED TRAINING TIME ESTIMATE:")
    print(f"   • Training images: {train_images} (vs 338 original)")
    print(f"   • Data increase: {((train_images/338)-1)*100:.0f}% more training data")
    print(f"   • Epochs: {epochs}")
    print(f"   • Batch size: {batch_size}")
    print(f"   • Image size: {img_size}px")
    print(f"   • Time per epoch: ~{epoch_time/60:.1f} minutes")
    print(f"   • Total time: ~{total_hours:.1f} hours")
    print(f"   • Expected accuracy: 97-98% (vs 89-95% with small dataset)")
    
    return total_hours

estimated_time = estimate_laptop_training_time()

if estimated_time <= 12:
    print(f"\n✅ REALISTIC TRAINING TIME!")
    print(f"🎯 Perfect for overnight training - much higher accuracy expected!")
else:
    print(f"\n⚠️ Still long but worth it for 98% accuracy target")

💻 Starting laptop-optimized training...
Target: 90-92% accuracy in 6-8 hours
⚡ Much faster than original 17-hour configuration

⏱️ OPTIMIZED TIME ESTIMATE:
   • Training images: 338
   • Epochs: 80 (vs 120)
   • Batch size: 4
   • Image size: 416px (vs 640px)
   • Time per epoch: ~2.5 minutes
   • Total time: ~3.4 hours
   • Expected accuracy: 90-92%

✅ REALISTIC TRAINING TIME!
🎯 Perfect for overnight or weekend training


In [5]:
# OPTIMIZED TRAINING
print("Starting training...")

try:
    results = model.train(
        # Dataset configuration
        data='datasets/palms_yolo/dataset.yaml',
        
        # OPTIMIZED: Reduced training time
        epochs=80,                              
        patience=15,                             # Reduced patience
        
        # OPTIMIZED: Faster processing
        imgsz=416,                             # Reduced from 640
        batch=4,                                # Increased from 2 (better efficiency)
        
        # OPTIMIZED: Simpler training
        lr0=0.01,                               # Standard learning rate
        optimizer='SGD',                        # Faster than AdamW on CPU
        cos_lr=False,                           # Disable cosine LR for speed
        warmup_epochs=3,                        # Reduced warmup
        
        # OPTIMIZED: Lighter augmentation 
        hsv_h=0.01,                            # Reduced augmentation
        hsv_s=0.4,                             # Reduced augmentation
        hsv_v=0.2,                             # Reduced augmentation
        degrees=5,                             # Reduced rotation
        translate=0.05,                        # Reduced translation
        scale=0.3,                             # Reduced scale
        fliplr=0.5,                            # Keep horizontal flip
        mosaic=0.8,                            # Reduced mosaic
        mixup=0.0,                             # Disable mixup for speed
        
        # Laptop optimization
        name='Palm_Training',     # Experiment name
        save=True,                             # Save checkpoints
        save_period=20,                        # Save less frequently
        plots=True,                            # Generate plots
        
        # Hardware settings
        device='cpu',                          # Force CPU (explicit)
        workers=4,                             # Reduced workers for stability
        
        # Memory optimization
        val=True,                              # Enable validation
        cache=False,                           # No caching (saves RAM)
        verbose=True,                          # Show progress
    )
    
    print("\n🎉 Laptop-optimized training completed!")
    print(f"📊 Results saved in: runs/detect/{results.save_dir}")
    
    # Show key metrics
    print(f"\n📈 Training Summary:")
    print(f"   • Total epochs run: {len(results.metrics) if hasattr(results, 'metrics') else 'N/A'}")
    print(f"   • Training completed successfully!")
    
except Exception as e:
    print(f"❌ Training error: {e}")
    print("\n💡 Troubleshooting:")
    print("   1. Ensure dataset paths are correct")
    print("   2. Check available disk space")
    print("   3. Try reducing batch size to 2")
    print("   4. Consider running with even fewer epochs (60)")

print("\n🎯 Expected Results:")
print("   • Training time: ~6-8 hours")
print("   • Expected accuracy: 90-92%") 
print("   • Good balance of speed vs performance for laptops")

Starting training...
New https://pypi.org/project/ultralytics/8.3.217 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.161  Python-3.10.0 torch-2.7.1+cpu CPU (11th Gen Intel Core(TM) i3-1115G4 3.00GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/palms_yolo/dataset.yaml, degrees=5, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01, hsv_s=0.4, hsv_v=0.2, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.8, multi_scale=False, name=  Palm_Training2

train: Scanning C:\Users\anish\OneDrive\Desktop\FYP\fyp-palm\datasets\palms_yolo\train\labels.cache... 338 images, 0 backgrounds, 0 corrupt: 100%|██████████| 338/338 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.10.0 ms, read: 292.686.9 MB/s, size: 153.1 KB)


c:\Users\anish\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\anish\OneDrive\Desktop\FYP\fyp-palm\datasets\palms_yolo\valid\labels.cache... 22 images, 0 backgrounds, 0 corrupt: 100%|██████████| 22/22 [00:00<?, ?it/s]
c:\Users\anish\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to runs\detect\  Palm_Training2\labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 416 train, 416 val
Using 0 dataloader workers
Logging results to runs\detect\  Palm_Training2
Starting training for 80 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/80         0G       1.81      2.961      2.051          6        416: 100%|██████████| 85/85 [01:17<00:00,  1.10it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.29it/s]

                   all         22         31     0.0048          1      0.156     0.0679



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/80         0G      1.285      2.637      1.568         13        416:  78%|███████▊  | 66/85 [00:52<00:15,  1.27it/s]


KeyboardInterrupt: 